## Spatial Tessellations

For GPS data it is often useful to create spatial tessellations.

This practically means that you are creating a grid of squares or hexagons and data is mapped into these squares. It is much easier to analyse data in such manner as it discretizes real-valued GPS locations into integer locations. Integer locations are way simpler to use in prediction models.

In this notebook:

    1) Create a tessellation of Beijing
    2) Mapping of data points into tessellation
    3) Create an animation of GPS logging taxis throughout day - count of taxis in each rectangle/hexagon.
    
    
Note!!

- Since the data is from 2008, the background map might not totally match the GPS data! There might be new roads, buildings destroyed or built, etc..

In [1]:
import sys
sys.path.append("../") # Add parent dir to source
import pandas as pd
import numpy as np

import folium
from folium import plugins
from folium.plugins import HeatMap
import datetime
import math


pd.set_option('mode.chained_assignment', None)

In [2]:
df = pd.read_pickle("../processed_data/fully_processed.pkl")

In [3]:
df.head()

,taxi_id,time,long,lat
188,1,2008-02-04 10:45:00,116.53787,39.91500
189,1,2008-02-04 10:55:00,116.50839,39.90916
190,1,2008-02-04 11:05:00,116.50940,39.90679
191,1,2008-02-04 17:28:00,116.47511,39.91397
192,1,2008-02-04 17:38:00,116.51175,39.91550


## Creating a Tessellation

In order to discretize longitudes and latitudes it is useful to create a tessellation. Lets start with basic rectangular grid.

        0 1 2  
        2 3 4
        5 6 7
        
We will split Beijing up into some sized rectangles. Here we will do it in a stupid simple way.. Just creating rectangles based on coords and not considering Earth's curvature! Therefore the actual real-life areas of these rectangles are not equal! But I guess it is good enough for now...

In [4]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Source: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [5]:
# Explanation of these variables is in notebook 3. In short, these variables describe the boundary box for Beijing.
# All of our data is in this box.
min_lat = 39.76158
max_lat = 40.03603
min_long = 116.19549
max_long = 116.56628

diff_lat = max_lat - min_lat
diff_long = max_long - min_long


print("diff_lat", diff_lat)
print("diff_long", diff_long)


diff_lat 0.27444999999999453
diff_long 0.3707899999999995


Lets try breaking it down into 25x25 grid..



In [6]:
import geopandas
from shapely.geometry import Polygon
from geopandas import GeoSeries

# Define mapping function. Map coordinate into grid slot

class RectangularTessellation:
    """
    Class for creating a tessellation. 
    Contains functions for mapping points into a tessellation.
    """
    
    def __init__(self, min_lon, min_lat, max_lon, max_lat, size: int):
        self.min_lon = min_lon
        self.min_lat = min_lat
        self.max_lon = max_lon
        self.max_lat = max_lat
        self.size = size
        assert min_lon < max_lon
        assert min_lat < max_lat
        
        ## Create linspaces and meshgrid, probably will not need
        self.x_linspace = np.linspace(min_lon, max_lon, num=self.size+1)
        self.y_linspace = np.linspace(min_lat, max_lat, num=self.size+1)
    
        # Figure out stepsizes for boxes. This we need to map to bo
        self.x_stepsize = self.x_linspace[1] - self.x_linspace[0]
        self.y_stepsize = self.y_linspace[1] - self.y_linspace[0]
        
    
    def get_polygons(self):
        """
        Creates geopandas Poly objects. Returns a series of it.
        The order of creation is from smallest long to max long (left-to-right)
                                and inner loop is smallest lat to max lat (down-to-up)
        
        
        Therefore the order of grid is
        
        3 6 9
        2 5 8
        1 4 7
        """
        polygons = []
        for i in range(len(self.x_linspace)-1):
            for j in range(len(self.y_linspace)-1):
                #print(len(polygons), i, j, i*self.size+j)
                poly = Polygon([(self.x_linspace[i], self.y_linspace[j]),
                                (self.x_linspace[i+1], self.y_linspace[j]),
                                (self.x_linspace[i+1], self.y_linspace[j+1]),
                                (self.x_linspace[i], self.y_linspace[j+1])
                               ])
                polygons.append(poly)
                
        g = GeoSeries(polygons)
        
        return g
    
    def map_to_grid(self, lon, lat):
        """
        Maps latitude and longitude to created meshgrid.
        f(lon, lat) -> grid_i, grid_j
        """
        #print("lon_diff", lon - self.min_lon)
        #print("lat_diff", lat - self.min_lat)

        
        x_grid = np.int64((lon - self.min_lon) / self.x_stepsize)
        y_grid = np.int64((lat - self.min_lat) / self.y_stepsize)
        
        ## HACK!! Fix edge case.. Some points are in size+1'th box
        x_grid = np.clip(x_grid, a_min=0, a_max=self.size-1)
        y_grid = np.clip(y_grid, a_min=0, a_max=self.size-1)
            
        return x_grid, y_grid
    
    def calculate_occurrences_by_grid_slot(self, df):
        """
        df: input data with mapped already columns grid_x and grid_y
        """
        assert "grid_x" in df.columns and "grid_y" in df.columns
        
        occurrences_by_grid = df.groupby(["grid_x", 
                                          "grid_y"]).count().reset_index()[["grid_x", "grid_y", "taxi_id"]]
        
        occurrences_by_grid = occurrences_by_grid.rename(columns={"taxi_id": "count"})


        # Add missing counts. If not all boxes are filled.
        index_ = pd.MultiIndex.from_product([[i for i in range(self.size)], 
                                             [i for i in range(self.size)]], 
                                            names=["grid_x", "grid_y"])
        
        occurrences_by_grid = occurrences_by_grid.set_index(
            ["grid_x", "grid_y"]).join(pd.DataFrame(index=index_), how="right").fillna(0)
        
        occurrences_by_grid["count"] = occurrences_by_grid.astype(int)
        occurrences_by_grid = occurrences_by_grid.reset_index()

        # Get grid nr! it is grid_y (row) * grid_size + grid_x
        occurrences_by_grid["grid_nr"] = (
            occurrences_by_grid["grid_y"]*self.size + occurrences_by_grid["grid_x"]).astype("str")


        return occurrences_by_grid



In [7]:
grid_size = 25
tess = RectangularTessellation(min_lon=min_long, 
                               min_lat=min_lat, 
                               max_lon=max_long,
                               max_lat=max_lat,
                               size=grid_size) # size = number of rectangles


In [8]:
# Tests
#print(tess.map_to_grid(min_long, min_lat) == (0,0))
#print(tess.map_to_grid(max_long, max_lat) == (grid_size-1, grid_size-1)) # Fails, division inaccuracies. 
#print(tess.map_to_grid(min_long, max_lat) == (0, grid_size-1)) # fails, division inaccuracies
#print(tess.map_to_grid(max_long, min_lat) == (grid_size-1, 0))


### Grid ready

Now we have a function to map coordinates into pairs (x,y). Defined by rectangles. Now we can map all our spatial data coordinates into these rectangles.

### Visualize tessallation

Lets the tessallation using Folium.

In [9]:
m_points = folium.Map(location=[39.907, 116.438], zoom_start=10, tiles="Stamen Toner")
m_points.fit_bounds([[max_lat, max_long], [min_lat, min_long]])

folium.Choropleth(
    geo_data=tess.get_polygons().to_json(),
    name="choropleth",
    fill_color="YlGn",
    fill_opacity=0.3,
    line_opacity=0.5,
).add_to(m_points)


m_points

### Map all data to the tessellation

Map all the data to tessellation and plot it using Folium

In [10]:
# Map all data points to grid
tess = RectangularTessellation(min_lon=min_long, 
                               min_lat=min_lat, 
                               max_lon=max_long,
                               max_lat=max_lat,
                               size=grid_size)
df["grid_x"], df["grid_y"] = tess.map_to_grid(df["long"], df["lat"])
df.head()


,taxi_id,time,long,lat,grid_x,grid_y
188,1,2008-02-04 10:45:00,116.53787,39.91500,23,13
189,1,2008-02-04 10:55:00,116.50839,39.90916,21,13
190,1,2008-02-04 11:05:00,116.50940,39.90679,21,13
191,1,2008-02-04 17:28:00,116.47511,39.91397,18,13
192,1,2008-02-04 17:38:00,116.51175,39.91550,21,14


In [11]:
occurrences_by_grid = tess.calculate_occurrences_by_grid_slot(df)

In [12]:
m_points = folium.Map(location=[39.907, 116.438], zoom_start=10, tiles="Stamen Toner")
m_points.fit_bounds([[max_lat, max_long], [min_lat, min_long]])

folium.Choropleth(
    geo_data=tess.get_polygons().to_json(),
    name="choropleth",
    data=occurrences_by_grid,
    columns=["grid_nr", "count"],
    key_on="feature.id",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="Count of occurrences",
).add_to(m_points)


m_points

### Repeat for 50x50 grid



In [13]:
occurrences_by_grid

,grid_x,grid_y,count,grid_nr
0,0,0,1140,0
1,0,1,17,25
2,0,2,162,50
3,0,3,556,75
4,0,4,1992,100
...,...,...,...,...
620,24,20,1215,524
621,24,21,554,549
622,24,22,1001,574
623,24,23,965,599


In [14]:
#df_copy = df.copy()
#df_copy["long"] = min_long
#df_copy["lat"] = max_lat

In [15]:
# Map all data points to grid

grid_size = 50
tess = RectangularTessellation(min_lon=min_long, 
                               min_lat=min_lat, 
                               max_lon=max_long,
                               max_lat=max_lat,
                               size=grid_size)


df["grid_x"], df["grid_y"] = tess.map_to_grid(df["long"], df["lat"])
#df_copy["grid_x"] = 0
#df_copy["grid_y"] = 2
occurrences_by_grid = tess.calculate_occurrences_by_grid_slot(df)



#occurrences_by_grid.loc[5, "count"] = 100000000

m_points = folium.Map(location=[39.907, 116.438], zoom_start=10, tiles="Stamen Toner")
m_points.fit_bounds([[max_lat, max_long], [min_lat, min_long]])

folium.Choropleth(
    geo_data=tess.get_polygons().to_json(),
    name="choropleth",
    data=occurrences_by_grid,
    columns=["grid_nr", "count"],
    key_on="feature.id",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Count of occurrences",
).add_to(m_points)


m_points

### Visualize Monday's activity

Create an animation. For every half hour visualize current locations of taxis. For each taxi pick the earliest location for that half-hour.

As I haven't totally figured out GeoJson and timestamped GeoJson, I'll do it the raw way.. Create images and create .gif..

Used Selenium webdriver and command line convert..

    convert -delay 100 -loop 0 $(ls -v1 *.png) monday.gif


In [16]:
# For each select first measurement.
monday_df = df[(df["time"].dt.day==4)]


In [17]:
# For each taxi, pick one measurement 30 min
first_each_half_hour = monday_df.set_index("time").groupby(["taxi_id", pd.Grouper(freq="30min")]).first().reset_index()

In [18]:
grid_size = 25
tess = RectangularTessellation(min_lon=min_long, 
                               min_lat=min_lat, 
                               max_lon=max_long,
                               max_lat=max_lat,
                               size=grid_size)

all_occurrences_by_time = []
time_stamp = []
for k, v in first_each_half_hour.groupby("time"):
    v["grid_x"], v["grid_y"] = tess.map_to_grid(v["long"], v["lat"])
    occs = tess.calculate_occurrences_by_grid_slot(v)
    occs["time"] = k
    all_occurrences_by_time.append(occs)
    time_stamp.append(k)
    
    
time_occs = pd.concat(all_occurrences_by_time)

# Get maximum
max_count_active_once = time_occs["count"].max()
# Round up to next 100.
max_count_active_once -= max_count_active_once % -100
max_count_active_once

200

In [19]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from PIL import Image
import io

def run_selenium_generation_1():

    driver = webdriver.Chrome(ChromeDriverManager().install())

    for i, v in enumerate(all_occurrences_by_time):
        m = folium.Map(tiles="Stamen Toner")

        title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(f"Monday {str(time_stamp[i])}")  


        m.fit_bounds([[max_lat, max_long], [min_lat, min_long]])

        nbins = 10
        step = math.ceil(max_count_active_once//nbins+1)
        # create bins
        bins = [step*i for i in range(nbins)]

        folium.Choropleth(
            geo_data=tess.get_polygons().to_json(),
            name="choropleth",
            data=all_occurrences_by_time[i],
            columns=["grid_nr", "count"],
            key_on="feature.id",
            fill_color="YlGnBu",
            fill_opacity=0.7,
            line_opacity=0.2,
            bins=bins,
            legend_name="Taxi count",

        ).add_to(m)

        m.get_root().html.add_child(folium.Element(title_html))

        img_data = m._to_png(5)
        img = Image.open(io.BytesIO(img_data))
        img.save(f"monday_image_{i+1}.png")

#run_selenium_generation_1()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/home/joonas/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


### End result


<img src="monday.gif" width="1000" align="center">


We can see that most of the activity is in the center of the city. Which should be logical. We also see some anomaly at top right corner, with many taxies logging GPS locations here.

Looking up that location on map, I found out that it looks something similar to a train station? Or it just has many railroads. Which would make sense as many taxis would be waiting for arrivals there. It is also interesting that it disappears during the daytime - probably due to the fact that not many taxis are not waiting there anymore.

Other possible reasoning is that it is just a taxi depot... 


### Repeat creation of plot, but only include active/moving taxi

As was done in second notebook, I'll now only include taxis which have moved more than 1 km past 30 min.


In [20]:
df[["long_prev", "lat_prev", "time_prev"]] = df.groupby('taxi_id')[['long', 'lat', "time"]].shift()
df["distance_from_prev"] = haversine_np(df["long"], df["lat"], df["long_prev"], df["lat_prev"])

taxi_moving_activity = df.set_index("time").groupby(
    ["taxi_id", pd.Grouper(freq="30min")]
).sum()

ids_active_taxis = taxi_moving_activity[taxi_moving_activity["distance_from_prev"] > 1].index

# Keep these intervals with Haversine distance sum more than 1 km

taxi_moving_activity = df.set_index("time").groupby(
    ["taxi_id", pd.Grouper(freq="30min")]
).first()


In [21]:
# Keep only active taxis. Take first sample from each of them? 
#taxi_moving_activity = taxi_moving_activity.reset_index()
active_taxis_first = taxi_moving_activity[taxi_moving_activity.index.isin(set(ids_active_taxis))].reset_index()

#ids_active_taxis

In [22]:
monday_df_2 = active_taxis_first[(active_taxis_first["time"].dt.day==4)]

first_each_half_hour_2 = monday_df_2.set_index("time").groupby(["taxi_id", pd.Grouper(freq="30min")]).first().reset_index()

In [23]:
grid_size = 25
tess = RectangularTessellation(min_lon=min_long, 
                               min_lat=min_lat, 
                               max_lon=max_long,
                               max_lat=max_lat,
                               size=grid_size)

all_occurrences_by_time = []
time_stamp = []
for k, v in first_each_half_hour_2.groupby("time"):
    v["grid_x"], v["grid_y"] = tess.map_to_grid(v["long"], v["lat"])
    occs = tess.calculate_occurrences_by_grid_slot(v)
    occs["time"] = k
    all_occurrences_by_time.append(occs)
    time_stamp.append(k)
    
time_occs = pd.concat(all_occurrences_by_time)

# Get maximum
max_count_active_once = time_occs["count"].max()
# Round up to next 100.
max_count_active_once -= max_count_active_once % -100
max_count_active_once

200

In [24]:
def run_selenium_generation_2():
    driver = webdriver.Chrome(ChromeDriverManager().install())

    for i, v in enumerate(all_occurrences_by_time):
        m = folium.Map(tiles="Stamen Toner")

        title_html = '''
                 <h3 align="center" style="font-size:16px"><b>{}</b></h3>
                 '''.format(f"Monday, Active taxi heuristic: {str(time_stamp[i])}")  


        m.fit_bounds([[max_lat, max_long], [min_lat, min_long]])

        nbins = 10
        step = math.ceil(max_count_active_once//nbins+1)
        # create bins
        bins = [step*i for i in range(nbins)]

        folium.Choropleth(
            geo_data=tess.get_polygons().to_json(),
            name="choropleth",
            data=all_occurrences_by_time[i],
            columns=["grid_nr", "count"],
            key_on="feature.id",
            fill_color="YlGnBu",
            fill_opacity=0.7,
            line_opacity=0.2,
            bins=bins,
            legend_name="Taxi count",

        ).add_to(m)

        m.get_root().html.add_child(folium.Element(title_html))

        img_data = m._to_png(5)
        img = Image.open(io.BytesIO(img_data))
        img.save(f"monday_active_image_{i+1}.png")

#run_selenium_generation_2()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/home/joonas/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


### Result


<img src="monday_active.gif" width="1000" align="center">


We can still notice that most of the activity is around the center of the city. Less activity during the night, more during the day.

In early morning after 3 am, there is almost no activity. When morning starts to rise, we see activity pattern between the center of the city and train station (and also towards airport, which is though, out of this grid, but there is movement towards north-east).

    
    
## Discussion


Basic tessellation of rectangles might not be good enough. Some rectangles here are always empty or almost. 
It does capture some patterns however showing which parts of the city are more active.
    
    